# LexisNexis 기사 데이터 정제

## 1. 라이브러리 불러오기

In [1]:
import pandas as pd
import numpy as np
import spacy
import stanza
import os
import re
from tqdm import tqdm
# Stanza 초기화
stanza.download('id')  # 영어 모델 다운로드

# 스탄자 인도네시아어 모델 로드
stanza_id = stanza.Pipeline('id', processors='tokenize')

# 스파이시 영어 모델 로드
nlp = spacy.load('en_core_web_sm')

2023-11-22 10:19:08 INFO: Downloading default packages for language: id (Indonesian) ...
2023-11-22 10:19:09 INFO: File exists: C:\Users\USER\stanza_resources\id\default.zip
2023-11-22 10:19:13 INFO: Finished downloading models and saved to C:\Users\USER\stanza_resources.
2023-11-22 10:19:13 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2023-11-22 10:19:14 WARNING: Language id package default expects mwt, which has been added
2023-11-22 10:19:14 INFO: Loading these models for language: id (Indonesian):
| Processor | Package |
-----------------------
| tokenize  | gsd     |
| mwt       | gsd     |

2023-11-22 10:19:14 INFO: Using device: cpu
2023-11-22 10:19:14 INFO: Loading: tokenize
2023-11-22 10:19:14 INFO: Loading: mwt
2023-11-22 10:19:14 INFO: Done loading processors!


## 2. 함수 정의
##### - 파일 읽기
##### - 데이터 전처리 및 변환
##### - 메타데이터 설정

In [2]:
def read_file(file_name):
    global date, keyword
    
    # 파일명에서 날짜와 도시 이름을 추출하는 정규표현식 패턴
    pattern = re.compile(r'(\d{8})_newsdata_(.*)\.csv')
    match = re.match(pattern, file_name)

    if match:
        date = match.group(1)  # 추출한 날짜
        keyword = match.group(2)  # 추출한 도시 이름
        print("Date:", date)
        print("Keyword:", keyword)
    else:
        print("No match found")


    df= pd.read_csv(file_name)
    return df


#수집일 및 키워드 표기
def set_metadata(df):

    date_str = date

    # YYYY-MM-DD 형태로 수집 날짜 명시
    formatted_date = f"{date[:4]}-{date[4:6]}-{date[6:]}"
    print("collection date :", formatted_date)

    # 파일명에서 확장자(.csv)를 제외한 부분 선택
    prefix = file_name.split('.')[0]
    
    df['Doc_ID'] = prefix + "_" + df['ID'].apply(format_id).astype(str)
    df['Col_Date']= formatted_date
    df['Keyword']=keyword
    
    for idx in df.index :
        if type(df['Date'][idx]) == float:
            # Check if it's the first row
            if idx == df.index[0]:
                # Handle the first row differently. For example, set it to NaN or a default date.
                df['Date'][idx] = pd.NaT
            else:
                df['Date'][idx] = df['Date'][idx-1]



def cleansing(text):
    # "MMM DD, YYYY" 형식의 날짜와 ")"로 끝나는 부분을 찾아 제거하는 정규표현식
    pattern_1 = r'\w{3}\s\d{2},\s\d{4}\(.*?\)'
    pattern_2 = r'Loading Link to the original story\. Notes.*?sole discretion\.'
    pattern_3 = r'\b[A-Z][a-z]{2} \d{2}, \d{4} \(.*?by Newstex\)'
    
    # 정규표현식을 사용하여 패턴을 제거
    cleaned_text = re.sub(pattern_3, '', text, flags=re.DOTALL)
    
    # cleaned_text = re.sub(pattern_1, '', text)

    # 왼쪽 공백 제거
    cleaned_text = cleaned_text.lstrip()

    if text.endswith('sole discretion.'):
        cleaned_text= cleaned_text[:-1284]
    

    # 날짜 형식 제거 (예: Apr 08, 2013)
    cleaned_text = re.sub(r'\b[A-Z][a-z]{2} \d{2}, \d{4}\b', '', cleaned_text)
    # http:// 또는 https:// 형태 제거
    cleaned_text= re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', cleaned_text)
    # 대괄호 안 숫자 제거
    cleaned_text = re.sub(r'\[\d+\]', '', cleaned_text)
    # <--> 형태 제거
    cleaned_text = re.sub(r'<.*?>', '', cleaned_text)
    

    # 정규표현식 패턴을 사용하여 해당 패턴 제거
    pic_pattern = re.compile(r'pic\.twitter\.com\/[A-Za-z0-9]+')
    cleaned_text = re.sub(pic_pattern, "", cleaned_text)


    # 한글 제거
    text_without_korean = re.sub('[가-힣]+', '', cleaned_text)
    ko_pattern = re.compile(r'[ㄱ-ㅎㅏ-ㅣ]+')
    text_without_korean = re.sub(ko_pattern, '', text_without_korean)
    
    
    # 링크 제거 (http:// 또는 https://로 시작하는 링크)
    text_without_links = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', text_without_korean)
    if (text_without_links.startswith("(")) & (")" in text_without_links):
        end_index = text_without_links.index(')')
        text_without_links = text_without_links[end_index+1:]
    if text_without_links.startswith("(KoreanIndo: Delivered by Newstex)"):
        text_without_links = text_without_links[len("(KoreanIndo: Delivered by Newstex)"):]
    
    
    pattern = re.compile(r'(source|Source)')
    match = pattern.search(text_without_links)

    
    if match:
        text_without_links = re.split(r'(source|Source).*', text_without_links)[0].strip()
        
    else:
        pass
    text_without_links = text_without_links.replace("Loading", "")
    
    
    text_without_links = text_without_links.replace("[embedded content]", "")
    text_without_links = re.sub(r'\[ \d+\]:', '', text_without_links)
    return text_without_links


def cleansing_text(test):

    test = re.sub(r"Baca Juga:.*?\.", "", test) #Baca Juga 관련 문장 제거
    test = re.sub(r"Baca Juga :.*?\.", "", test)
    test = re.sub(r"Baca juga:.*?\.", "", test)
    test = re.sub(r"Baca juga.*?\.", "", test)
    test = re.sub(r"Baca juga :.*?\.", "", test)
    test = re.sub(r"baca juga:.*?\.", "", test)
    test = re.sub(r"baca juga :.*?\.", "", test)
    test = re.sub(r"baca Juga:.*?\.", "", test)
    test = re.sub(r"baca Juga :.*?\.", "", test)
    test = re.sub(r"Foto:.*?\.","",test) #Foto 관련 문장 제거
    test = re.sub(r"Written by:.*","",test) #Written by: 이후 모든 문장 제거
    test = re.sub(r"Written by :.*","",test)
    test = re.sub(r"written by:.*","",test)
    test = re.sub(r"written by :.*","",test)
    test = re.sub(r"written by .*","",test)
    test = re.sub(r"—- Written by :.*","",test)
    test = re.sub(r"—Written by :.*","",test)
    test = re.sub(r"\b[A-Z][a-z]{2} \d{2}, \d{4} \(.*?by Newstex\)","",test) #날짜 제거
    test = re.sub(r"http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+","",test) #링크 제거
    test = re.sub(r"\[\d+\]","",test) #대괄호 포함 내용 제거
    test = re.sub(r"\[ \d+\]","",test) #대괄호 포함 내용 제거
    test = re.sub(r":","",test)
    test = re.sub(r"<.*?>","",test)
    test = re.sub(r"//instagram.com/.*","",test) #//instagram.com/ 이후 모든 문장 제거
    test = re.sub(r"\b(?:\w+\W+){0,30}Soompi(?:\W+\w+){0,30}\b", "", test) #Soompi 기준 앞 뒤 단어 30개 제거
    test = re.sub(r"\b(?:\w+\W+){0,30}soompi(?:\W+\w+){0,30}\b","",test) #soompi
    test = re.sub(r'#\d+;', "", test) ##8216;, #8217; 등 제거
    test = re.sub(r'# \d+;', "", test)
    test = re.sub(r'#\d+ ;', "", test)
    test = re.sub(r'# \d+ ;', "", test)
        
    return test


def second_cleansing_text(dtest):

    dtest = re.sub(r"\[ \d+\]","",dtest)
    dtest = re.sub(r"Written by:.*","",dtest) #Written by: 이후 모든 문장 제거
    dtest = re.sub(r"Written by :.*","",dtest)
    dtest = re.sub(r"written by:.*","",dtest)
    dtest = re.sub(r"written by :.*","",dtest)
    dtest = re.sub(r"//instagram.com/.*","",dtest) #//instagram.com/ 이후 모든 문장 제거
    dtest = re.sub(r"\b(?:\w+\W+){0,30}Soompi(?:\W+\w+){0,30}\b", "", dtest) #Soompi 기준 앞 뒤 단어 30개 제거
    dtest = re.sub(r"\b(?:\w+\W+){0,30}soompi(?:\W+\w+){0,30}\b","",dtest) #soompi
    dtest = re.sub(r":","",dtest)
    dtest = re.sub(r"[一-龥]+","",dtest)

    return dtest


# 1 -> 000001, 10231 -> 010231 형태로 id number 바꾸기
def format_id(id_number):
    # Check if the value is NaN
    if pd.isna(id_number):
        return 'NaN'
    
    formatted_id = '{:06d}'.format(int(id_number))
    return formatted_id


# 'Article' Column에 있는 기사 본문의 metadata 명시 및 불필요 텍스트 제거 후 'Text' Column으로 생성
def preprocessing(df) :
    set_metadata(df)
    df['Text'] = df['Article'].apply(cleansing_text)
    df['Text'] = df['Text'].apply(cleansing)
    df['Text'] = df['Text'].apply(second_cleansing_text)

## 3. 데이터프레임 형태로 폴더 내부 원시데이터 파일 합치기

In [11]:
path = 'C:\\Users\\USER\\Desktop\\test'
os.chdir(path)

In [1]:
# 현재 폴더 내의 파일 리스트 얻기
file_list = os.listdir()

# 폴더 내 파일들에 대해 반복하여 CSV 파일 읽기
df_all= pd.DataFrame()
new_df = pd.DataFrame()

for file_name in tqdm(file_list):
    if file_name.endswith('.csv'):
        new_df = read_file(file_name)
        print(f'filename : {file_name}')
        print(f"length : {len(new_df)}")
        preprocessing(new_df)

    df_all =pd.concat([df_all, new_df], axis=0)
print("-----------------------------------\n")

# news_list에 있는 키워드가 들어간 요소들 추리기
news_list= ['KoreanIndo', 'Newstex Blogs', 'The Conversation (Indonesia Edition)', 'Mobile88.com', 'GlobeNewswire Indonesian',
             'ParsToday (Indonesian)', 'Plus Company Updates(PCU)', 'Mongabay.com - Indonesian', 'Thomson Reuters ONE',
               'Koreana', 'Webnews - Indonesian', 'Berita Harian', 'ParsToday (Indonesian)', 'Berita Dalam Negeri',
                 'TendersInfo - Contract Awards', 'Antara News', 'Industry SnapShot',
                   'Plus Company Updates(PCU)', 'Berita Ekonomi', 'ASAPII Database', 'Channel NewsAsia', 'BruDirect',
                     'US Official News' , 'Nepal Weekly', 'The New York Times', 'Premium Official News',
                      'Media OutReach Newswire (Indonesian)', 'US Fed News', 'States News Service', 'CNN Indonesia', 'M2 PressWIRE', 
                        'Asia News Monitor', 'Global English (Middle East and North Africa Financial Network)', 'Indonesia Tribune', 
                          'News Direct', 'MENAFN -Press Releases (English)', 'Contify Insurance News', 'Islamic Finance Monitor Worldwide', 'Indonesia Government News', 'ICT Monitor Worldwide'
                            ]
df_all= df_all[df_all['Newspaper'].isin(news_list)]

# 중복 제거 후 날짜 기준으로 정렬
df_all.drop_duplicates(subset='Text', inplace=True)
df_all.sort_values('Date', inplace=True)
df_all.reset_index(drop=True, inplace=True)

df_all

NameError: name 'os' is not defined

## 4. 토크나이징 함수 정의
##### - 'Text' 문장 분리 진행 후 'Sentence' 열에 분리된 문장 나열
##### - 'Sentence' 열의 인도네시아 단어들을 전처리 및 토큰화 진행 후 'Token'열에 나열 

In [16]:
def tokenizing(df):
    new_rows = []
    max_length = 5
    for index, row in tqdm(df.iterrows(), total=df.shape[0], desc="Processing rows"):
        article_text = row['Text']
        doc = nlp(article_text)
        sen_id = 1  # 문장 ID를 1로 초기화

        for sentence in doc.sents:  # spacy에서는 .sents로 문장에 접근
            sentence = re.sub(r'Here.*website:', "", sentence.text)
            sentence = re.sub(r'shared from .*', "", sentence.text)
            sentence = re.sub(r'Shared from .*', "", sentence)
            sentence = re.sub(r'shared by .*', "", sentence)
            sentence = re.sub(r'Shared by .*', "", sentence)
            sentence = re.sub(r'&#\d+;', "", sentence) #remove &#38;(문장 부호 유형)
            sentence = re.sub(r'#\d+;', "", sentence) #remove #8216;, #8217; etc(문장 부호 유형)
            sentence = re.sub(r'# \d+;', "", sentence)
            sentence = re.sub(r'#\d+ ;', "", sentence)
            sentence = re.sub(r'# \d+ ;', "", sentence)
            sentence = re.sub(r"'#\d+;[^']+'", "", sentence)
            sentence = re.sub(r"ig_mid=[A-Z0-9\-]+", "", sentence) # #38; 뒤에 오는 ig_mid= 형태의 문장 제거(문장 부호 유형)
            sentence = re.sub(r'\[ \d+ \] : .*',"",sentence) #remove [ 1 ] : [ 2 ] : [ 3 ] etc
            sentence = re.sub(r"Written by:.*","",sentence) #Written by: 이후 모든 문장 제거(출처&링크 유형)
            sentence = re.sub(r"Written by :.*","",sentence)
            sentence = re.sub(r"Written by.*","",sentence)
            sentence = re.sub(r"written by:.*","",sentence)
            sentence = re.sub(r"written by :.*","",sentence)
            sentence = re.sub(r"written by .*","",sentence)
            sentence = re.sub(r"—- Written by :.*","",sentence)
            sentence = re.sub(r"—Written by :.*","",sentence)
            sentence = re.sub(r"#main #main #main.*","",sentence) # #main 이후 모든 문장 제거
            sentence = re.sub(r"About these ads.*","",sentence) #About these ads. 이후 모든 문장 제거(광고 유형)
            sentence = re.sub(r"Allkpop.*","",sentence) #Allkpop 이후 모든 문장 제거(제거 시 가끔 이메일 주소도 같이 삭제)
            sentence = re.sub(r"allkpop.*","",sentence)
            sentence = re.sub(r"Indotrans.*","",sentence) #allkpop 바로 뒤에 나오는 indotrans 이후 모든 문장 제거
            sentence = re.sub(r"indotrans.*","",sentence)
            sentence = re.sub(r"\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.(com|net|kr)\b","",sentence) #remove email(.com, .net, .kr)
            sentence = re.sub(r"\([^@]+@[^\)]+\)","",sentence) #(@e-mail) 형태 문장 제거
            sentence = re.sub(r"\(@[^)]+\)","",sentence) #(@단어) 형태 문장 제거(SNS 계정 포함)
            sentence = re.sub(r"\( @[^)]+\)","",sentence) 
            sentence = re.sub(r"\(#[^)]+\)","",sentence) #(#단어) 형태 문장 제거
            sentence = re.sub(r"@\w+\b","",sentence) #@포함 바로 뒤에 오는 단어 제거
            sentence = re.sub(r"#\w+\b","",sentence) # #포함 바로 뒤에 오는 단어 제거
            sentence = re.sub(r"utm_medium.*","",sentence) #utm_medium 이후 모든 문장 제거 (Soompi 관련 text)
            sentence = re.sub(r"utm_campaign.*","",sentence)
            sentence = re.sub(r"Photo Credit.*","",sentence) #Photo Credit/credit 이후 모든 문장 재거
            sentence = re.sub(r"Photo credit.*","",sentence)
            sentence = re.sub(r"Article credit.*","",sentence)
            sentence = re.sub(r"Lihat videonya.*","",sentence) #광고문 뒷문장 모두 제거
            sentence = re.sub(r"lihat videonya.*","",sentence) 
            sentence = re.sub(r"^.*\d+\s*points\s*\d+.*$","",sentence) #'숫자 points 숫자' 형태의 문장 제거(고유명사 나열 유형)
            sentence = re.sub(r"^.*\d+\s*Points\s*\d+.*$","",sentence) 
            sentence = re.sub(r"^.*\d+\s*points\s*\.*$","",sentence)
            sentence = re.sub(r"^.*\d+\s*poin.*$","",sentence) # '숫자 poin' 형태의 문장 제거(고유명사 나열 유형)
            sentence = re.sub(r"^.*\d+\s*Poin.*$","",sentence)
            sentence = re.sub(r"^.*berikut\sini\.\s*$","",sentence) #텍스트 마지막에 존재하는 berikut ini. 가 포함된 문장 제거(인니어 광고문 유형)
            sentence = re.sub(r"^.*bawah ini.*$","",sentence) #di bawah ini가 포함된 문장 제거(인니어 광고문 유형)
            sentence = re.sub(r"^.*\.\.\.$","",sentence) # ...으로 끝나는 문장 제거(문장 잘림 유형)
            sentence = re.sub(r"\[.*?\]","",sentence) # 대괄호 및 단어 제거
            sentence = re.sub(r"www\.[a-zA-Z0-9-]+\.com","",sentence) # www.com 형태의 문장 제거
            sentence = re.sub(r"\b\w+\.com\b","",sentence) # .com 형태의 문장 제거
            sentence = re.sub(r"\([^)]*persen[^)]*\)","",sentence) #(persen...text) 형태의 문장 제거
            sentence = re.sub(r"\([^)]*Entertainment[^)]*\)","",sentence) #(Entertainment...text) 형태의 문장 제거
            sentence = re.sub(r".+\s*\.\.\.$","",sentence) #마지막이 ...로 끝나는 문장 모두 제거
            sentence = re.sub(r".+\s*….$","",sentence)
            sentence = re.sub(r"\(.*?\)","",sentence) #()포함 소괄호 안에 있는 텍스트까지 제거

            #하나씩 제거하는 정규표현식
            
            sentence = re.sub(r'gt;_lt;', '',sentence) #remove ;gt and ;lt
            sentence = re.sub(r'gt;',"",sentence)
            sentence = re.sub(r'lt;',"",sentence)
            sentence = re.sub(r'_lt;',"",sentence)
            sentence = re.sub(r"Advertisements","",sentence) #remove english
            sentence = re.sub(r"Instagrammable","",sentence)
            sentence = re.sub(r"instagrammable","",sentence)
            sentence = re.sub(r"LOL","",sentence)
            sentence = re.sub(r"LoL","",sentence)
            sentence = re.sub(r"gantiin","",sentence) #remove wrong spelling
            sentence = re.sub(r"content]","",sentence)
            sentence = re.sub(r"@ star1","",sentence) #@star1 이라는 잡지회사 제거
            sentence = re.sub(r"@star1","",sentence)
            sentence = re.sub(r"♥","",sentence) #특수문자 제거
            sentence = re.sub(r"▲","",sentence)
            sentence = re.sub(r"=","",sentence)
            sentence = re.sub(r"#","",sentence) 
            sentence = re.sub(r"@","",sentence) 
            sentence = re.sub(r"\[","",sentence) 
            sentence = re.sub(r"]","",sentence) 
            sentence = re.sub(r"~","",sentence) 
            sentence = re.sub(r"\^","",sentence) 
            sentence = re.sub(r"→","",sentence) 
            sentence = re.sub(r"-","",sentence)
            spacy_doc = nlp(sentence)  # Renamed to spacy_doc for clarity
            tokens = [token.text for sent in spacy_doc.sents for token in sent]
            
            # 문장, 토큰, 문장길이, Sen_ID 컬럼 추가
            new_row = row.copy()
            new_row['Sentence'] = sentence
            new_row['Token'] = tokens
            new_row['Sen_len'] = len(tokens)
            new_row['Sen_ID'] = f"{row['Doc_ID']}_sen{sen_id:06d}"
            
            new_rows.append(new_row)
            sen_id += 1  # 문장 ID 증가


    # 리스트 기호 없이 단어 단위로 토큰화된 새로운 데이터 프레임 생성
    new_df = pd.DataFrame(new_rows)
    # 토큰 길이 필터링
    new_df = new_df[new_df['Token'].apply(lambda x: len(x) > max_length)]

    new_df['Tokenized_Sentence']= new_df['Token'].apply(' '.join)
    new_df['Pub_Type']='Newspaper'

    # 'Sentence' 컬럼 중심으로 중복 제거
    new_df.drop_duplicates(subset='Sentence', inplace=True)
    new_df.reset_index(drop=True, inplace=True)

    # 공식 제출용 컬럼명으로 컬럼명 변환
    new_df.rename(columns={'Source_File' : 'Filename', 'Headline':'Title', 'Keyword':'Pub_Subj', 'Date': 'Pub_Date', 'Sen_len': 'Word_Count'}, inplace=True)
    new_df = new_df[['Doc_ID', 'Filename', 'Title', 'Pub_Type', 'Pub_Subj', 'Pub_Date', 'Col_Date', 'Sen_ID', 'Word_Count', 'Text', 'Sentence','Tokenized_Sentence' , 'Token']]
    
    # 특정 단어나 구문 제거
    words_to_remove = ['/instagram', 'Written by', 'written by', 'WRITTEN BY', 'POWERED BY',
                    'ads', 'full credits', 'Full Credits', 'FULL CREDITS']
    new_df['Tokenized_Sentence'] = new_df['Tokenized_Sentence'].apply(remove_sentences_with_words, words=words_to_remove)
    new_df['Tokenized_Sentence'].replace('', np.nan, inplace=True)
    new_df.dropna(inplace=True)
    
    return new_df


def remove_sentences_with_words(text, words):
    if not text:
        return ""  # 만약 텍스트가 비어있으면 빈 문자열을 반환

    # 간단한 정규 표현식을 사용하여 텍스트를 문장으로 분리
    sentences = re.split(r'(?<=[.!?])\s+', text)

    # 필터링된 문장을 저장하기 위한 리스트 초기화
    filtered_sentences = []

    for sentence in sentences:
        # 주어진 단어들 중 하나라도 문장에 존재하는지 확인 (대소문자 구분 없음)
        if any(re.search(fr'\b{re.escape(word)}\b', sentence, re.IGNORECASE) for word in words):
            continue  # 만약 특정 단어를 포함하는 문장이면 건너뜀
        else:
            filtered_sentences.append(sentence)  # 그렇지 않은 경우, 문장을 리스트에 추가

    # 필터링된 문장들을 하나의 텍스트로 결합
    result = ' '.join(filtered_sentences)
    
    return result  # 결과 반환

##### Start Tokenizing

In [18]:
df = tokenizing(df_all)
df

Processing rows:   0%|          | 0/269 [00:00<?, ?it/s]

Processing rows: 100%|██████████| 269/269 [04:08<00:00,  1.08it/s]


,Doc_ID,Filename,Title,Pub_Type,Pub_Subj,Pub_Date,Col_Date,Sen_ID,Word_Count,Text,Sentence,Tokenized_Sentence,Token
0,20230920_newsdata_Koreana2_000001,C:/Users/USER/Desktop/news data/20230920_data/...,Mempertahankan Rasa “Kelambanan”,Newspaper,Koreana2,2020-01-01,2023-09-20,20230920_newsdata_Koreana2_000001_sen000001,12,Fermentasi yang memerlukan waktu dan perawatan...,Fermentasi yang memerlukan waktu dan perawatan...,Fermentasi yang memerlukan waktu dan perawatan...,"[Fermentasi, yang, memerlukan, waktu, dan, per..."
1,20230920_newsdata_Koreana2_000001,C:/Users/USER/Desktop/news data/20230920_data/...,Mempertahankan Rasa “Kelambanan”,Newspaper,Koreana2,2020-01-01,2023-09-20,20230920_newsdata_Koreana2_000001_sen000003,32,Fermentasi yang memerlukan waktu dan perawatan...,Di antara sejumlah makanan fermentasi Korea ya...,Di antara sejumlah makanan fermentasi Korea ya...,"[Di, antara, sejumlah, makanan, fermentasi, Ko..."
2,20230920_newsdata_Koreana2_000001,C:/Users/USER/Desktop/news data/20230920_data/...,Mempertahankan Rasa “Kelambanan”,Newspaper,Koreana2,2020-01-01,2023-09-20,20230920_newsdata_Koreana2_000001_sen000004,25,Fermentasi yang memerlukan waktu dan perawatan...,Satu keluarga secara khusus telah mempertahank...,Satu keluarga secara khusus telah mempertahank...,"[Satu, keluarga, secara, khusus, telah, memper..."
3,20230920_newsdata_Koreana2_000001,C:/Users/USER/Desktop/news data/20230920_data/...,Mempertahankan Rasa “Kelambanan”,Newspaper,Koreana2,2020-01-01,2023-09-20,20230920_newsdata_Koreana2_000001_sen000005,16,Fermentasi yang memerlukan waktu dan perawatan...,“Kami membuat meju dengan merebus kacang pada ...,“ Kami membuat meju dengan merebus kacang pada...,"[“, Kami, membuat, meju, dengan, merebus, kaca..."
4,20230920_newsdata_Koreana2_000001,C:/Users/USER/Desktop/news data/20230920_data/...,Mempertahankan Rasa “Kelambanan”,Newspaper,Koreana2,2020-01-01,2023-09-20,20230920_newsdata_Koreana2_000001_sen000006,19,Fermentasi yang memerlukan waktu dan perawatan...,"Setelah difermentasi selama sebulan, kami baru...","Setelah difermentasi selama sebulan , kami bar...","[Setelah, difermentasi, selama, sebulan, ,, ka..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21972,20230920_newsdata_Koreana2_000270,C:/Users/USER/Desktop/news data/20230920_data/...,Trashbusters: Gerakan Lingkungan sebagai Budaya,Newspaper,Koreana2,2023-04-01,2023-09-20,20230920_newsdata_Koreana2_000270_sen000072,18,Trashbusters tidak putus asa saat memandang tu...,Era yang memungkinkan apa saja dapat diantar m...,Era yang memungkinkan apa saja dapat diantar m...,"[Era, yang, memungkinkan, apa, saja, dapat, di..."
21973,20230920_newsdata_Koreana2_000270,C:/Users/USER/Desktop/news data/20230920_data/...,Trashbusters: Gerakan Lingkungan sebagai Budaya,Newspaper,Koreana2,2023-04-01,2023-09-20,20230920_newsdata_Koreana2_000270_sen000073,23,Trashbusters tidak putus asa saat memandang tu...,Meski penggunaan kantong plastik sudah dilaran...,Meski penggunaan kantong plastik sudah dilaran...,"[Meski, penggunaan, kantong, plastik, sudah, d..."
21974,20230920_newsdata_Koreana2_000270,C:/Users/USER/Desktop/news data/20230920_data/...,Trashbusters: Gerakan Lingkungan sebagai Budaya,Newspaper,Koreana2,2023-04-01,2023-09-20,20230920_newsdata_Koreana2_000270_sen000074,8,Trashbusters tidak putus asa saat memandang tu...,Halhal itu mungkin dapat memicu tawa sinis.,Halhal itu mungkin dapat memicu tawa sinis .,"[Halhal, itu, mungkin, dapat, memicu, tawa, si..."
21975,20230920_newsdata_Koreana2_000270,C:/Users/USER/Desktop/news data/20230920_data/...,Trashbusters: Gerakan Lingkungan sebagai Budaya,Newspaper,Koreana2,2023-04-01,2023-09-20,20230920_newsdata_Koreana2_000270_sen000075,9,Trashbusters tidak putus asa saat memandang tu...,"Meskipun demikian, mari kita mengucapkan sloga...","Meskipun demikian , mari kita mengucapkan slog...","[Meskipun, demikian, ,, mari, kita, mengucapka..."


##### 테스트웍스(가공 담당 회사) 전달 전 각 Token에 대한 column 생성

In [ ]:
# 토큰별 작업용 컬럼 생성
for i in range(df['Word_Count'].max()):
    new_column_name = f'Column_{i}'  # 새 열 이름 생성
    df[new_column_name]=np.nan

for idx in tqdm(df.index):
    for i, token in enumerate(df['Token'][idx]):
        new_column_name = f'Column_{i}'  # 새 열 이름 생성
        df[new_column_name][idx]=token

In [ ]:
# 'Sen_ID'기준으로 정렬
df.sort_values('Sen_ID', inplace=True)
df.reset_index(drop=True, inplace=True)
df

## 5. pickle 형태로 정제 완료 데이터 저장

In [ ]:
# 날짜_newsdata_원천데이터 합본.pickle
df.to_pickle('20231111_news_assemble_unique.pickle')